## Элементы функционального программирования в Python
* [Чистые функции](#pure_functions)
* Имутабельность
* Функции, как объекты первого класса
    * lambda
    * частичное применение
    * карирование
    * [композиция](#composition)
* Ленивые вычисления
* map, filter
* itertools, functools, operator
* Примеры задач

In [1]:
import copy
import collections
import datetime
import functools
import itertools
import operator

import toolz
from functional import seq as seq_


<a id=’pure_functions’></a>
## Чистые функции

Чистые функции зависят только от своих параметров и возвращают только свой результат. Следующая функция вызванная несколько раз с одним и тем же аргументом выдаст разный результат.

In [2]:
# Чистые функции

def is_new_year() -> bool:
    today = datetime.date.today()
    return today.day == 31 and today.month == 12

is_new_year()


False

In [3]:
def pure_is_new_year(month: int, day: int) -> bool:
    return month == 12 and day == 31

pure_is_new_year(12, 31)


True

In [15]:
# Чистые функции - имутабельность

l = [1, 3, 2]

def replace_first_element_and_sort(element: int, l: list[int]) -> list[int]:
    l[0] = element
    l.sort()
    return l

changed_l = replace_first_element_and_sort(5, l)
print('Impure')
print(l)
print(changed_l)

Impure
[2, 3, 5]
[2, 3, 5]


In [16]:
l = [1, 3, 2]

def pure_replace_first_element_and_sort(element: int, l: list[int]) -> list[int]:
    # l = l[:]
    l = copy.deepcopy(l)
    l[0] = element
    return list(sorted(l))

changed_l = pure_replace_first_element_and_sort(5, l)
print('Pure')
print(l)
print(changed_l)


Pure
[1, 3, 2]
[2, 3, 5]


###  1st class object

In [17]:
def foo_func():
    print('I am foo')
    
bar_func = foo_func
bar_func()


func_list = [foo_func, bar_func]
func_list[1]()
d = {foo_func: 'foo'}
print(d[foo_func])

I am foo
I am foo
foo


In [50]:
def function_creator():
    def inner():
        print('inner')
        
    return inner

created_function = function_creator()
created_function()

def function_creator_v2(f):
    
    def inner():
        f()
        print('inner_v2')
    
    return inner

function_creator_v2(created_function)()
    
    

inner
inner
inner_v2


In [8]:
def is_a(char: str) -> bool:
    return char == 'a'


print(sorted('abbaab', key=is_a, reverse=True))

['a', 'a', 'a', 'b', 'b', 'b']


#### lambda

In [29]:
def lambda_example(a, b, c):
    return a + b + c

lambda_example = lambda a, b, c: a + b + c

print(sorted('abbaab', key=lambda c: c == 'a', reverse=True))

['a', 'a', 'a', 'b', 'b', 'b']


In [19]:
def my_filter(pred, seq):
    result = []
    for x in seq:
        if pred(x):
            result.append(x)
    return result

# Переиспользование
seq = [1, 0, 2]
non_zero = my_filter(bool, seq)
only_odd = my_filter(lambda x: x % 2, seq)
only_even = my_filter(lambda x: not x % 2, range(10))

### Частичное применение
Это процесс фиксации части аргументов функции

In [23]:
from functools import partial

# filter(pred, seq)
filter_bool = partial(my_filter, bool)

list(filter_bool([0, 1, None, 2, None]))
list(filter_bool([2, None]))

foo = partial(filter_bool, [0, 1, None, 2, None])
list(foo())

[1, 2]

In [25]:
#  f - частичное применение

l = [1, 3, 2]


def pure_replace_n_element_to(n: int, to: int, l: list[int]) -> list[int]:
    l = l[:]
    l[n] = to
    return l


pure_replace_first_element_to_100 = functools.partial(pure_replace_n_element_to, n=0, to=100)

changed_l = pure_replace_first_element_to_100(l=l)
print(changed_l)


print(list(reversed(pure_replace_n_element_to(n=0, to=100, l=l))))
print(list(reversed(pure_replace_first_element_to_100(l=l))))
    
    


[100, 3, 2]
[2, 3, 100]
[2, 3, 100]


#### переиспользование

In [26]:
from functools import partial

def is_even(number):
    return number % 2 == 0

filter_even = partial(my_filter, is_even)
list(filter_even(range(10)))

[0, 2, 4, 6, 8]

#### переиспользование

In [14]:
from itertools import filterfalse

filter_odd = partial(filterfalse, is_even)
list(filter_odd(range(10)))

[1, 3, 5, 7, 9]

### map, filter...

In [55]:
# urls = ['http://www.google.com', 'http://www.wikipedia.com', 'http://www.apple.com']
# html_texts = []
# for item in urls:
#     html_texts.append(urlopen(item))
# return html_texts


# integers = [1, 2, 3, 4, 5]
# fib_integers = []
# for item in integers:
#     fib_integers.append(fib(item))
# return fib_integers


print(list(map(lambda x, y: f'{x} {y}', [1, 2, 3], [4, 5, 6])))

def is_even(x):
    return (x % 2) == 0

print(list(filter(is_even, range(10))))

['1 4', '2 5', '3 6']
[0, 2, 4, 6, 8]


<a id=’composition’></a>
### Композиция

In [35]:
#  f - композиция функций
def compose2(f, g):
    return lambda x: f(g(x))

def double(x):
    return x * 2

def inc(x):
    return x + 1

# def inc_and_double(x):
#     inced = inc(x) 
#     doubled_x = double(inced)
#     return doubled_x

inc_and_double = compose2(double, inc)

print(inc_and_double(10))

def compose(*fns):
    init, *rest = reversed(fns)
    
    def inner(*args, **kwargs):
        result = init(*args, **kwargs)
        for fn in rest:
            result = fn(result)
        return result
    
    return inner
    
# Теперь мы можем делать всякие штуки (выполнение идет справа налево):
mapv = compose(list, map)


filterv = compose(list, filter)


filterv(bool, range(3))

map_str = partial(map, str)
awesome = compose(list, map_str, filter)
awesome(bool, range(10))

# mapv(str, range(10)) == list(map(str, range(10)))


22


['1', '2', '3', '4', '5', '6', '7', '8', '9']

#### Каррирование

In [32]:
#  f - каррирование (синтаксический сахар для partial)

@toolz.curry
def mul(x, y):
    return x * y

# mul_by_5 = partial(mul, 5)
mul_by_5 = mul(5)


mul_by_5(2)

10

In [8]:
pure_replace_n_element_to_curried = toolz.curry(pure_replace_n_element_to)
pure_replace_n_element_to_curried(1)(100)([1, 2, 3])
pure_replace_first_element_to_100(l=[1, 2, 3])

NameError: name 'pure_replace_n_element_to' is not defined

#### Ленивые вычисления

##### Генераторы

In [42]:
def my_filter(pred, seq):
    for x in seq:
        if pred(x):
            yield x
            
            
def even_generator(max):
    n = 2
    while n <= max:
        result = n
        n += 2
        yield result

g = even_generator(4)
# print([i for i in g])
# print(g)
# print(next(g))
# print(next(g))
# print(next(g)) StopIteration


def heavy_operation1(x):
     print('heavy_operation1')
     return x + 1

def heavy_operation2(x):
     print('heavy_operation2')
     return x + 2

g = map(heavy_operation1, map(heavy_operation2, [1, 2 ,3]))
next(g)
next(g)

heavy_operation2
heavy_operation1
heavy_operation2
heavy_operation1


5

##### Итераторы

In [26]:
class EvenIterator:

    def __init__(self, end):
        self.current_n = 2
        self.end = end

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_n <= self.end:
            result = self.current_n
            self.current_n += 2
            return result

        raise StopIteration

print([i for i in EvenIterator(10)])


iterator = iter([1, 2])
print(iterator.__next__())
print(next(iterator))
# print(next(iterator))  StopIteration

[2, 4, 6, 8, 10]
1
2


### functools, itertools, operator
The operator module was mentioned earlier. It contains a set of functions corresponding to Python’s operators. These functions are often useful in functional-style code because they save you from writing trivial functions that perform a single operation.

In [45]:
add_5 = toolz.curry(operator.add)(5)
list(map(add_5, [1, 2, 3]))

[6, 7, 8]

## Примеры задач


### Написать функцию, которая считает кол-во каждого слова в тексте

In [65]:
 
def stem(word: str) -> str:
     """ Stem word to primitive form """
     return word.lower().rstrip(",.!:;'-\"").lstrip("'\"")

def wordcount_imperative(text: str) -> dict[str, int]:
    word_count = collections.defaultdict(int)
    words = text.split()
    
    for word in words:
        stemed_word = stem(word)
        word_count[stemed_word] += 1
        
    return dict(word_count)

In [63]:
def wordcount_func(text):
    return toolz.frequencies(map(stem, text.split()))

In [66]:
text = 'a a b bb a a A BB b'
print(wordcount_func(text))
print(wordcount_imperative(text))

{'a': 5, 'b': 2, 'bb': 2}
{'a': 5, 'b': 2, 'bb': 2}


In [67]:
wordcount_func = toolz.compose(toolz.frequencies, toolz.curried.map(stem), str.split)

wordcount_func(text)

{'a': 5, 'b': 2, 'bb': 2}

## Получить первые пять чисел, которые делятся без остатка на 100, превратить в строки.

In [48]:
def get_first_5_imperative():
    result = []
    for i in range(1_000_000):
        if (i % 100) == 0:
            result_str = f'Number is {i}'
            result.append(result_str)

        if len(result) == 5:
            break
    return result

print(get_first_5_imperative())



['Number is 0', 'Number is 100', 'Number is 200', 'Number is 300', 'Number is 400']


In [47]:
source = range(1_000_000)

def get_first_5_functional(source):
    return itertools.islice(
        map(
            lambda num: f'Number is {num}',
            filter(
                lambda num: num % 100 == 0,
                source,
            )
        ),
        0,
        5,
    )

print(list(get_first_5_functional(source)))



['Number is 0', 'Number is 100', 'Number is 200', 'Number is 300', 'Number is 400']


In [12]:
def format_number(number: int) -> str:
    return f'Number is {number}'


def divided_by_number(number: int, n: int) -> bool:
    return n % number == 0

divided_by_100 = functools.partial(divided_by_number, 100)

def get_first_5_functional(source):
    return itertools.islice(map(format_number, filter(divided_by_100, source)),0,5)

print(list(get_first_5_functional(source)))

['Number is 0', 'Number is 100', 'Number is 200', 'Number is 300', 'Number is 400']


In [68]:
list(toolz.compose(toolz.curried.take(5), toolz.curried.map(format_number), toolz.curried.filter(divided_by_100))(source))

['Number is 0',
 'Number is 100',
 'Number is 200',
 'Number is 300',
 'Number is 400']

### Композиция по-другому

In [49]:
seq_(range(1_000_000))\
    .filter(divided_by_100)\
    .map(lambda x: x + 5)\
    .map(format_number)\
    .slice(0, 5)


['Number is 5', 'Number is 105', 'Number is 205', 'Number is 305', 'Number is 405']